In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import docker
import time
import concurrent.futures
from datetime import datetime
import copy
import math
from collections import defaultdict
import pprint
from collections import namedtuple
from datetime import datetime
import os
import ctypes as ct
import multiprocessing
import shutil

In [4]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
# !!! Is not used because it missed every container running under 1 second.
# Run docker-activity to monitor power consumption and runtime of the containers.

# Initialize Docker client
client = docker.from_env()
activity_container = client.containers.run(
    image="jdrouet/docker-activity",
    command=["stdout"],
    volumes={
        "/sys/class/powercap": {"bind": "/sys/class/powercap", "mode": "ro"},
        "/var/run/docker.sock": {"bind": "/var/run/docker.sock", "mode": "rw"},
    },
    privileged=True,
    detach=True,
    auto_remove=True
)
for log in activity_container.logs(stream=True, follow=True):
    print(log.decode().strip())

In [ ]:
# Does not work here...
# Run the ebpf-energy-monitoring container to capture the benchmarks metrics.
# Initialize Docker client
client = docker.from_env()
os.makedirs("output", exist_ok=True)

container = client.containers.run(
    image="ebpf-mon",
    name="ebpf-mon",
    privileged=True,
    cap_add=["SYS_ADMIN", "SYS_PTRACE"],
    security_opt=["seccomp=unconfined", "apparmor=unconfined"],
    volumes={
        "/lib/modules": {"bind": "/lib/modules", "mode": "ro"},
        "/usr/src": {"bind": "/usr/src", "mode": "ro"},
        "/etc/localtime": {"bind": "/etc/localtime", "mode": "ro"},
        "/sys/kernel/debug": {"bind": "/sys/kernel/debug", "mode": "rw"},
        "/proc": {"bind": "/host/proc", "mode": "ro"},
        os.path.join(os.getcwd(), "config.yaml"): {"bind": "/home/config.yaml", "mode": "rw"},
        "/var/run/docker.sock": {"bind": "/var/run/docker.sock", "mode": "rw"},
        os.path.join(os.getcwd(), "output"): {"bind": "/output", "mode": "rw"},
    },
    network_mode="host",
    detach=True,
    remove=True,
    stdin_open=True,
    tty=True,
    command="/bin/bash"  # This matches your CLI command
)

for line in container.logs(stream=True, stdout=True, stderr=True, follow=True):
    print(line.decode().rstrip())

In [ ]:
# Run isolated benchmarks in a Docker container on the different cores.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    # Ensure start_time is set even if the container does not reach 'running'
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
            
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    isolated_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=800000000000","run"], "cpuset_cpus": "1"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=50G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()  

In [2]:
# Uses linpack | Run isolated benchmarks in a Docker container on the different cores.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    if label is None:
        label = next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels = {"test": label},
        **run_kwargs
    )
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    # Ensure start_time is set even if the container does not reach 'running'
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
            
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    isolated_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="4",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "0", "label":"cpu", "environment": {"LINPACK_ARRAY_SIZE": "10000"}},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=1T", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=500G","--file-block-size=4K","--file-test-mode=rndrw","--init-rng=on","--file-extra-flags=direct","--max-requests=0","run"],"cpuset_cpus": "4"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=1, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="4",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()  

Container started for Disk benchmark (prepare).
Container for benchmark ['linpack'] started successfully.
Container lifetime: 10.255128 seconds
Container on CPU 0 completed and removed.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=1M', '--memory-total-size=1T', '--threads=1', 'run'] started successfully.
Container lifetime: 10.164408 seconds
Container on CPU 2 completed and removed.
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=500G', '--file-block-size=4K', '--file-test-mode=rndrw', '--init-rng=on', '--file-extra-flags=direct', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.161008 seconds
Container on CPU 4 completed and removed.
Disk benchmark completed.
All isolated benchmarks completed.


In [4]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"


print("Container names and IDs:")
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in isolated_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  # Don't process contents of a deleted directory

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    isolated_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final isolated benchmarking results with power consumption:")
pprint.pprint(isolated_benchmarking_results)

Container names and IDs:
reverent_cerf: {'coloc_pair': None, 'workload': 'cpu', 'id': '060ed05c620ea1f3c6a99599ec92d970ff3641ead591dc22e22e86e0b6a73b39', 'life_time': 10.255128}
charming_yalow: {'coloc_pair': None, 'workload': 'memory', 'id': '2bf319e1c25b9552406f3e01a3fb95d99168a59516f5973c64481f9fa1686ce0', 'life_time': 10.164408}
strange_hugle: {'coloc_pair': None, 'workload': 'fileio', 'id': '46ca2a9b402d8396603af996ff9b1987142d3f8d8c97aefac83f7578819f15e7', 'life_time': 0.161008}
Final isolated benchmarking results with power consumption:
{'charming_yalow': {'coloc_pair': None,
                    'id': '2bf319e1c25b9552406f3e01a3fb95d99168a59516f5973c64481f9fa1686ce0',
                    'life_time': 10.164408,
                    'workload': 'memory'},
 'reverent_cerf': {'coloc_pair': None,
                   'id': '060ed05c620ea1f3c6a99599ec92d970ff3641ead591dc22e22e86e0b6a73b39',
                   'life_time': 10.255128,
                   'workload': 'cpu'},
 'strange_hugle

In [ ]:
# Run co-located benchmarks in a Docker container on the same core.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=300G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "0"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "24"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "25"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "2"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "26"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "3"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "27"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "4"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "28"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "5"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "29"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 

In [48]:
# Using linpack | Run co-located benchmarks in a Docker container on the same core physical core using logical pairs.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    if label is None:
        label = next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels = {"test": label},
        **run_kwargs
    )
    
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "0", "label":"cpu", "environment": {"LINPACK_ARRAY_SIZE": "10000"}},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=1T", "--threads=1", "run"], "cpuset_cpus": "24"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=1T", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=500G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "25"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "2", "label":"cpu", "environment": {"LINPACK_ARRAY_SIZE": "10000"}},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=500G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "26"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "23", "label": "cpu"},
            {"client": client, "image": "linpack", "command": ["linpack"], "cpuset_cpus": "47", "label": "cpu"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=1T", "--threads=1", "run"], "cpuset_cpus": "4"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=1T", "--threads=1", "run"], "cpuset_cpus": "28"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=500G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "5"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=500G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "29"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container for benchmark ['linpack'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=1T', '--threads=1', 'run'] started successfully.
Container lifetime: 10.19752 seconds
Container on CPU 0 completed and removed.
Container lifetime: 10.319299 seconds
Container on CPU 24 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=500G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.17356 seconds
Container on CPU 25 completed and removed.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=1T', '--threads=1', 'run'] started successfully.
Container lifetime: 10.482631 seconds
Container on C

In [49]:
# Access the watched benchmark containers for runtime and power consumption.
coloc_benchmarking_results_copy = copy.deepcopy(coloc_benchmarking_results)

print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")
# print(benchmarking_results)

Container names and IDs:
upbeat_borg: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'b9866dca617e60c3526f3b3cd59b3f815a2ca8cbcdad404a098f3b518c6e753a', 'colocated_runtime': 10.19752}
angry_wright: {'coloc_pair': 'CpuMem', 'workload': 'memory', 'id': 'c9ed0a277402a8c613bd9ddd60d9c16a920dbafe3e55191b81aa2a187d1c1db5', 'colocated_runtime': 10.319299}
wizardly_goldstine: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '5547aebe8959d752fbb7503bb60fb0d2cf383f9f25aae621d315802c5add32e0', 'colocated_runtime': 0.17356}
condescending_fermat: {'coloc_pair': 'MemFileIO', 'workload': 'memory', 'id': 'ece97eaa1c4ef6e9808e752e30c584915602bd01fbf5be3f87fc0552d9dcf5b6', 'colocated_runtime': 10.482631}
sad_wescoff: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': 'dbfb1a4d9dca9e122be376ed052ec3b311f753ce649bd4d895246494f0be6ddc', 'colocated_runtime': 0.216438}
hardcore_faraday: {'coloc_pair': 'FileIOCpu', 'workload': 'cpu', 'id': 'bb6bcdc4581a107c1c57daec8865ba5422a91f70dcbe118da98d2c

In [51]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"


print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in coloc_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    if container_name and container_name in coloc_benchmarking_results:
                        coloc_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final colocated benchmarking results with power consumption:")
pprint.pprint(coloc_benchmarking_results)

Container names and IDs:
upbeat_borg: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'b9866dca617e60c3526f3b3cd59b3f815a2ca8cbcdad404a098f3b518c6e753a', 'colocated_runtime': 10.19752, 'power': 15.037541182253557}
angry_wright: {'coloc_pair': 'CpuMem', 'workload': 'memory', 'id': 'c9ed0a277402a8c613bd9ddd60d9c16a920dbafe3e55191b81aa2a187d1c1db5', 'colocated_runtime': 10.319299, 'power': 12.500079190571663}
wizardly_goldstine: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '5547aebe8959d752fbb7503bb60fb0d2cf383f9f25aae621d315802c5add32e0', 'colocated_runtime': 0.17356}
condescending_fermat: {'coloc_pair': 'MemFileIO', 'workload': 'memory', 'id': 'ece97eaa1c4ef6e9808e752e30c584915602bd01fbf5be3f87fc0552d9dcf5b6', 'colocated_runtime': 10.482631, 'power': 0.8037262003705296}
sad_wescoff: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': 'dbfb1a4d9dca9e122be376ed052ec3b311f753ce649bd4d895246494f0be6ddc', 'colocated_runtime': 0.216438}
hardcore_faraday: {'coloc_pair': 'FileI

In [52]:
def calc_average_slowdown(slowdown_1, slowdown_2):
    average_slowdown = (slowdown_1 + slowdown_2) / 2
    return average_slowdown

def calc_slowdown_factor(isolated_runtime_1, isolated_runtime_2, coloc_runtime_1, coloc_runtime_2):
    slowdown_1 = isolated_runtime_1 / coloc_runtime_1
    slowdown_2 = isolated_runtime_2 / coloc_runtime_2
    average_slowdown = calc_average_slowdown(slowdown_1, slowdown_2)
    return average_slowdown

def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2):
    affinity_score = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
    return min(1, affinity_score)
    
for name, result in coloc_benchmarking_results.items():
    coloc_workload = result.get('workload')
    # Find the isolated runtime for the same workload
    isolated_runtime = ''
    isolated_power_consumption = ''
    for iso_name, iso_result in isolated_benchmarking_results.items():
        if iso_result.get('workload') == coloc_workload and iso_result.get('coloc_pair') is None:
            isolated_runtime = iso_result.get('life_time')
            isolated_power_consumption = iso_result.get('power')
            break
    result['isolated_runtime'] = isolated_runtime
    result['isolated_power_consumption'] = isolated_power_consumption
    print(f"Colocated benchmark: {coloc_workload}, {result.get('coloc_pair')}, Isolated runtime: {isolated_runtime}, Isolated power consumption: {isolated_power_consumption}")

Colocated benchmark: cpu, CpuMem, Isolated runtime: 10.188704, Isolated power consumption: 17.54964440182838
Colocated benchmark: memory, CpuMem, Isolated runtime: 10.158551, Isolated power consumption: 0.4783768060595573
Colocated benchmark: fileio, MemFileIO, Isolated runtime: 0.153357, Isolated power consumption: None
Colocated benchmark: memory, MemFileIO, Isolated runtime: 10.158551, Isolated power consumption: 0.4783768060595573
Colocated benchmark: fileio, FileIOCpu, Isolated runtime: 0.153357, Isolated power consumption: None
Colocated benchmark: cpu, FileIOCpu, Isolated runtime: 10.188704, Isolated power consumption: 17.54964440182838
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 10.188704, Isolated power consumption: 17.54964440182838
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 10.188704, Isolated power consumption: 17.54964440182838
Colocated benchmark: memory, MemMem, Isolated runtime: 10.158551, Isolated power consumption: 0.4783768060595573
Colocated benchma

In [53]:
# Transform benchmarking dict into a summary
coloc_summary = defaultdict(dict)

for name, result in coloc_benchmarking_results.items():
    pair = result['coloc_pair']
    workload = result.get('workload', 'unknown')
    if 'workload_1' not in coloc_summary[pair]:
        coloc_summary[pair]['workload_1'] = workload
        coloc_summary[pair]['colocated_runtime_1'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_1'] = result.get('isolated_runtime')
        coloc_summary[pair]['power_consumption_1'] = result.get('isolated_power_consumption')
    else:
        coloc_summary[pair]['workload_2'] = workload
        coloc_summary[pair]['colocated_runtime_2'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_2'] = result.get('isolated_runtime')
        coloc_summary[pair]['power_consumption_2'] = result.get('isolated_power_consumption')
        
# calculate and add average_slowdown and affinity_score for each coloc pair
for pair, summary in coloc_summary.items():
    try:
        iso1 = float(summary['isolated_runtime_1'])
        iso2 = float(summary['isolated_runtime_2'])
        coloc1 = float(summary['colocated_runtime_1'])
        coloc2 = float(summary['colocated_runtime_2'])
        summary['average_slowdown'] = calc_slowdown_factor(iso1, iso2, coloc1, coloc2)
        summary['affinity_score'] = calc_affinity_score(iso1, coloc1, iso2, coloc2)
    except Exception as e:
        summary['average_slowdown'] = None
        summary['affinity_score'] = None
        print(f"Could not calculate for pair {pair}: {e}")

coloc_summary = dict(coloc_summary)
pprint.pprint(coloc_summary)

{'CpuCpu': {'affinity_score': 0.9911813042346189,
            'average_slowdown': 0.9911850457231914,
            'colocated_runtime_1': 10.259383,
            'colocated_runtime_2': 10.299326,
            'isolated_runtime_1': 10.188704,
            'isolated_runtime_2': 10.188704,
            'power_consumption_1': 17.54964440182838,
            'power_consumption_2': 17.54964440182838,
            'workload_1': 'cpu',
            'workload_2': 'cpu'},
 'CpuMem': {'affinity_score': 0.9917353659941142,
            'average_slowdown': 0.9917790306851813,
            'colocated_runtime_1': 10.19752,
            'colocated_runtime_2': 10.319299,
            'isolated_runtime_1': 10.188704,
            'isolated_runtime_2': 10.158551,
            'power_consumption_1': 17.54964440182838,
            'power_consumption_2': 0.4783768060595573,
            'workload_1': 'cpu',
            'workload_2': 'memory'},
 'FileIOCpu': {'affinity_score': 0.9919034036414974,
               'average_sl

In [3]:
# Write affinity score matrix to a CSV file
affinity_score_matrix = pd.DataFrame.from_dict(coloc_summary, orient='index')
affinity_score_matrix = affinity_score_matrix.reset_index()
affinity_score_matrix.columns = ['coloc_pair', 'workload_1',  
                                  'workload_2', 'affinity_score']
# Write back to csv
affinity_score_matrix.to_csv('affinity_score_matrix.csv', index=False)

NameError: name 'coloc_summary' is not defined